In [7]:
import pandas as pd

# 读取两个文件
profile_df = pd.read_excel('IndividualInvestorProfile.xlsx')
user_df = pd.read_excel('User.xlsx')

# 按 user 分组聚合 profile_id（用逗号拼接）
profile_grouped = profile_df.groupby('user')['profile_id'].apply(
    lambda x: ', '.join(map(str, sorted(x)))
).reset_index()

# 重命名以便匹配
profile_grouped.rename(columns={'user': 'investor_profile_id'}, inplace=True)

# 用 user 表为主进行左连接
merged_df = pd.merge(user_df[['investor_profile_id']], profile_grouped, on='investor_profile_id', how='left')

# 调整列顺序：profile_id 在前，investor_profile_id 在后
result_df = merged_df[['profile_id', 'investor_profile_id']]

# 保存结果
result_df.to_excel('All_Users_with_ProfileIDs.xlsx', index=False)

print("处理完成，已保存为 All_Users_with_ProfileIDs.xlsx")


处理完成，已保存为 All_Users_with_ProfileIDs.xlsx


In [6]:
import pandas as pd

# === Step 1: 读取映射文件 ===
mapping_df = pd.read_excel('All_Users_with_ProfileIDs.xlsx')  # 含 profile_id, investor_profile_id, user_id

# 去除 profile_id 为 NaN 的行，分割成列表
mapping_df = mapping_df.dropna(subset=['profile_id'])
mapping_df['profile_id'] = mapping_df['profile_id'].astype(str).str.split(',')

# 展开成一行一个 profile_id，去空格，转 int
exploded_map = mapping_df.explode('profile_id')
exploded_map['profile_id'] = exploded_map['profile_id'].str.strip().astype(int)

# 得到反向映射：profile_id → user_id
reverse_mapping = exploded_map[['profile_id', 'user_id']]

# === Step 2: 读取原始 individualInvestorProfile 表 ===
profile_df = pd.read_excel('individualInvestorProfile.xlsx')  # 必须包含 profile_id 列

# === Step 3: 合并 user_id 到 profile_df ===
merged_df = profile_df.merge(reverse_mapping, on='profile_id', how='left')

# === Step 4: 保存结果 ===
merged_df.to_excel('individualInvestorProfile_with_user_id.xlsx', index=False)

print("✅ 已成功将 user_id 添加到 individualInvestorProfile.xlsx，并保存为 individualInvestorProfile_with_user_id.xlsx")


✅ 已成功将 user_id 添加到 individualInvestorProfile.xlsx，并保存为 individualInvestorProfile_with_user_id.xlsx


address 在bankaccount表中已经normalize了，替代掉user中的地址
问题：怎么对应上
办法：source and investor_name_raw in bankaccount join fund_id and investor_name_raw in individualInvestorProfile

In [21]:
import pandas as pd

# === Step 1: 读取数据 ===
user_df = pd.read_excel('User.xlsx')  # 包含 original_investor_profile_id
bank_df = pd.read_excel('BankAccount.xlsx')  # 包含 investor_name_raw 和地址字段

# === Step 2: 确保字段格式一致（统一为字符串去空格）===
user_df['original_investor_profile_id'] = user_df['original_investor_profile_id'].astype(str).str.strip()
bank_df['investor_name_raw'] = bank_df['investor_name_raw'].astype(str).str.strip()

# === Step 3: 合并 bankaccount 地址到 User ===
user_with_address = user_df.merge(
    bank_df[
        ['investor_name_raw',
         'bank_account_address_line_1',
         'bank_account_address_line_2',
         'bank_account_city',
         'bank_account_state',
         'bank_account_country',
         'bank_account_zipcode']
    ],
    left_on='original_investor_profile_id',
    right_on='investor_name_raw',
    how='left'
)

# （可选）删除多余的匹配列
user_with_address.drop(columns=['investor_name_raw'], inplace=True)

# === Step 4: 保存结果 ===
user_with_address.to_excel('User_with_BankAccount_Address.xlsx', index=False)

print("✅ BankAccount 地址已成功合并进 User 表并保存为 User_with_BankAccount_Address.xlsx")


✅ BankAccount 地址已成功合并进 User 表并保存为 User_with_BankAccount_Address.xlsx


In [22]:
import pandas as pd

# === Step 1: 读取数据 ===
user_df = pd.read_excel('User.xlsx')  # 包含 original_investor_profile_id
bank_df = pd.read_excel('BankAccount.xlsx')  # 包含 investor_name_raw 和地址字段

# === Step 2: 清洗字段，去空格、统一为小写以便匹配 ===
user_df['original_investor_profile_id'] = user_df['original_investor_profile_id'].astype(str).str.strip().str.lower()
bank_df['investor_name_raw'] = bank_df['investor_name_raw'].astype(str).str.strip().str.lower()

# === Step 3: 去除 bank_df 中重复的 investor_name_raw，只保留第一条地址 ===
bank_clean = bank_df.groupby('investor_name_raw').first().reset_index()

# === Step 4: 合并地址字段到 User 表 ===
address_fields = [
    'bank_account_address_line_1',
    'bank_account_address_line_2',
    'bank_account_city',
    'bank_account_state',
    'bank_account_country',
    'bank_account_zipcode'
]

user_with_address = user_df.merge(
    bank_clean[['investor_name_raw'] + address_fields],
    left_on='original_investor_profile_id',
    right_on='investor_name_raw',
    how='left'
)

# （可选）删除多余匹配列
user_with_address.drop(columns=['investor_name_raw'], inplace=True)

# === Step 5: 保存结果 ===
user_with_address.to_excel('User_with_BankAccount_Address.xlsx', index=False)

print("✅ 地址字段已成功合并并写入 User_with_BankAccount_Address.xlsx")


✅ 地址字段已成功合并并写入 User_with_BankAccount_Address.xlsx
